# Import

In [1]:
import pandas as pd
import plotly.graph_objs as go
import string
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#import cufflinks as cf
from plotly.offline import iplot
from tqdm import tqdm

from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
import plotly.express as px

#cf.go_offline()
#cf.set_config_file(offline = False, world_readable = True)

import plotly.io as pio
pio.renderers.default='notebook'

import warnings
warnings.simplefilter('ignore')

In [2]:
from rectools import Columns
from rectools.dataset import Dataset

In [3]:
import os
us = os.getcwd()
us

'/Users/liliyaivannikova/Documents/project/git/RecSys_Films'

In [4]:
if 'liliyaivannikova' in us:
    PATH = r'/Users/liliyaivannikova/Documents/project/ml-latest/'
    movies = pd.read_csv(PATH + r'movies.csv')
    rating = pd.read_csv(PATH + r'ratings.csv')
elif 'Владислав' in us:
    movies = pd.read_csv(r'dataset/movies.csv')
    rating = pd.read_csv(r'dataset/ratings.csv')

In [5]:
print(f'''
movies: {movies.shape}
rating: {rating.shape}
''')


movies: (86537, 3)
rating: (33832162, 4)



In [6]:
movies.columns = [col.upper() for col in movies.columns]
movies.head()

,MOVIEID,TITLE,GENRES
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies['REALEASE'] = movies['TITLE'].str.extract("\((\d{4})\)", expand=True)
movies['REALEASE'] = pd.to_datetime(movies['REALEASE'], format='%Y')
movies['REALEASE'] = movies['REALEASE'].dt.year
movies['TITLE'] = movies['TITLE'].str[:-7]

In [8]:
movies.head()

,MOVIEID,TITLE,GENRES,REALEASE
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


In [9]:
dfx = movies['GENRES'].str.get_dummies(sep='|')
for col in dfx.columns:
    dfx[col] = dfx[col].astype('int8')


In [10]:
movies = pd.concat([movies, dfx], axis=1) #.drop(columns=['GENRES'])
movies['TITLE'] = movies['TITLE'].astype('category')
movies['REALEASE'] = movies['REALEASE'].astype('float16')

In [11]:
movies['GENRES'] = movies['GENRES'].str.replace('|', ',')

In [12]:
movies.head()

,MOVIEID,TITLE,GENRES,REALEASE,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,"Adventure,Animation,Children,Comedy,Fantasy",1995.0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,"Adventure,Children,Fantasy",1995.0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,"Comedy,Romance",1995.0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,"Comedy,Drama,Romance",1995.0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,Comedy,1995.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
rating.columns = [col.upper() for col in rating.columns]
rating.head()

,USERID,MOVIEID,RATING,TIMESTAMP
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


# Обработка полей

In [14]:
rating['TIMESTAMP'] = pd.to_datetime(rating['TIMESTAMP'], unit='s')

In [15]:
rating.head()

,USERID,MOVIEID,RATING,TIMESTAMP
0,1,1,4.0,2008-11-03 17:52:19
1,1,110,4.0,2008-11-05 06:04:46
2,1,158,4.0,2008-11-03 17:31:43
3,1,260,4.5,2008-11-03 18:00:04
4,1,356,5.0,2008-11-03 17:58:39


<font color='red'> **Убрал записи  пустыми значениями ~3298, это около 0.01%, фигня короче, можно сносить.** </font>

In [16]:
rating.shape

(33832162, 4)

In [17]:
rating = rating[~((rating.TIMESTAMP.isna())|(rating.USERID.isna())|(rating.RATING.isna()))]

In [18]:
rating['USERID'].astype(str).value_counts().reset_index()[:5] 

,index,USERID
0,189614,33332
1,48766,9554
2,207216,9178
3,175998,9016
4,76618,8919


In [19]:
# при eda нашла странного юзера, уберем его при построении
print(rating.shape)
rating.query('USERID != 189614', inplace = True)
print(rating.shape)

(33832162, 4)
(33798830, 4)


# LightFM

In [20]:
from lightfm import LightFM
from lightfm.data import Dataset
import scipy.sparse as sp
from lightfm.evaluation import precision_at_k

In [21]:
def train_test(df, n_days):
    df['weight'] = np.where(df['RATING'] >= 4, 1, 0)
    train_df = df.loc[df['datetime'] < df['datetime'].max() - pd.DateOffset(days=n_days),
                     ['item_id', 'user_id', 'weight']].copy()
    test_df = df.loc[df['datetime'] >= df['datetime'].max() - pd.DateOffset(days=n_days),
                    ['item_id', 'user_id', 'weight']].copy()

    return train_df, test_df

In [22]:
class LightFM_ML():

    def __init__(self, model = None, df = None):
        if model is None:
            self.model = LightFM(learning_rate = 0.05,
                                 loss = 'warp', 
                                 random_state = 111)
        else:
            self.model = model
        
        self.df = df
        
    def IMDb(self):
        """
            Обработка кейсов, когда нет оценок по user_id
        """
        if self.df is None:
            return None
        else:
            pass
        # прописать IMDb
        
    def fit(self, df_train, epochs, item_features, num_threads):
        """
            Обучение модели LightFM
        """
        for i in tqdm(range(epochs)): 
            self.model.fit_partial(df_train, 
                            epochs = epochs, 
                            item_features = item_features,
                            num_threads = num_threads)
            
    def predict(self, user_id, k):
        """
            Рекомендации для пользователей
        """
        predictions = {}
        for user in user_id:
            predictions[user] = self.recommend(user_id=user, k=k)
        return predictions


    def recommend(self, user_id, k, movies_to_predict):
        """
            Расчет рекомендаций
        """
        # Если ранее такого пользователя не было, используем IMDb
        try:
            
            n_movies = self.df['item_id'].unique()

            prediction = self.model.predict(user_ids = user_id * len(movies_to_predict), 
                                        item_ids = movies_to_predict * len(user_id)
                                        )
            predict = np.array(n_movies)[np.argsort(-prediction)][:k]
            return predict.tolist()
        
        except:
            return self.IMDb()[:k]

In [23]:
def precision_k(model, df_test, k):
    test_precision = precision_at_k(model=model, 
                            test_interactions=df_test,
                            k=k).mean()
    return test_precision

In [24]:
rating.rename({'USERID':'user_id',
            'MOVIEID':'item_id',
            #'RATING':'weight',
            'TIMESTAMP':'datetime'}, axis=1, inplace=True)

In [25]:
movies.rename({'MOVIEID':'item_id'}, axis=1, inplace=True)

In [26]:
train_df, test_df = train_test(rating, 180)

In [27]:
dataset = Dataset()
dataset.fit(train_df['user_id'], train_df['item_id'])
dataset.fit_partial(users=test_df['user_id'], items=test_df['item_id'])

In [28]:
interactions, _ = dataset.build_interactions(train_df.loc[:, ['user_id', 'item_id']].values.tolist())
interactions_test, _ = dataset.build_interactions(test_df.loc[:, ['user_id', 'item_id']].values.tolist())

In [29]:
movies_to_predict = [dataset._item_id_mapping[x] for x in test_df['item_id'].unique()]

In [30]:
model = LightFM(no_components=10,
                loss='warp', 
                random_state=111,
                learning_rate=0.05)

lfm_model = LightFM_ML(model, rating)

In [31]:
# Обучение
lfm_model.fit(interactions, 
              epochs=5, 
              item_features = None,
              num_threads = 40)

100%|████████████████████████████████████████████| 5/5 [11:42<00:00, 140.43s/it]


In [32]:
predict = lfm_model.recommend(user_id = [22], 
                              k = 10, 
                              movies_to_predict = movies_to_predict)

In [33]:
predict

[3524, 2080, 79132, 4011, 1265, 209, 3361, 6534, 26184, 3186]

In [34]:
precision_k(model = lfm_model.model, df_test = interactions_test, k = 10)

0.12345693

In [35]:
precision_k(model = lfm_model.model, df_test = interactions, k = 10)

0.39999077